In [33]:
# transformers not support NumPy 2.0 yet
!pip install -q numpy~=1.26.4 transformers~=4.46.2

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


# Transformer 深入淺出

Google Research 和 Google Brain 的成員最初在 2017 年的論文《[Attention is all you need](https://arxiv.org/abs/1706.03762?context=cs)》中提出了 Transformer。目前是 NLP 中最受歡迎的架構之一。

Transformer 由 Encoder 和 Decoder 两部分组成，Encoder 接受 **输入** 然後生成 **向量** 表示這個输入，Decoder 接受 **向量** 然後逐一生成 **输出**。

![transformer in a nutshell](https://www.alexisalulema.com/wp-content/uploads/2022/08/encoder-decoder-1-1024x275.png)

Transformer 超越了基於 LSTM 和 RNN 的其他現有架構，獲得了更高評價和更快的訓練速度。Transformer 的注意力機制是一種「word-to-word」的操作，它會找出序列中每個詞與其他詞之間的關係，包括詞本身。

![Original Transformer Architecture, Source: Attention is all you need](https://www.alexisalulema.com/wp-content/uploads/2022/08/image-695x1024.png)

在本文中，我將以最簡單的方式解釋 Transformer 的架構和內部運作。

## 認識分詞器 (Tokenizer)

分詞器負責為模型準備輸入。大多數分詞器有兩種版本：完整的 Python 實現和基於 Rust 庫的“快速”實現。“快速”實現具有以下優點：

1. 特別是在批量分詞時顯著加速；

2. 提供額外的方法在原始字符串（字符和單詞）和標記空間之間進行映射（例如，獲取包含給定字符的標記索引或對應於給定標記的字符範圍）。

後面的課程會利用到這兩項優點，暫時無須深入了解。

分詞器 (Tokenizer) 通過將文本正規化來將輸入轉換為 token；此外，它還會提供一個整數表示（基於現有詞彙），該表示將用於 Embedding 過程。例如：

In [34]:
# 載入 BERT tokenizer
from transformers import AutoTokenizer
from pprint import pprint

model_name = "google-bert/bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

Tokenizater 包含以下重要資訊

* `vocab_size`：詞彙表的大小

* `model_max_length`：輸入到模型的最大標記數

* 特殊的 token，如

  * 0: [PAD]
  * 100: [UNK]
  * 101: [CLS]
  * 102: [SEP]
  * 103: [MASK]


In [35]:
# vocab_szie: 30522
# model_max_length: 512
tokenizer

BertTokenizerFast(name_or_path='google-bert/bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

### 字典 (Vocabulary)

顯示部分字典內容，從第 1000 個詞開始到 1010 個詞：

In [36]:
# 顯示挑選的 10 個字
pprint(list(tokenizer.vocab.keys())[1000:1010], compact=True)

['##port', 'hacked', '##ə', 'outgoing', 'autonomy', 'fk', 'parker', '##ikh',
 'cristina', 'pascal']


### 標記 (Token)

標記是分詞器的輸出，它是一個整數，代表字典中的一個詞。標記是模型的輸入。

In [37]:
raw_inputs = [
    "I've been waiting for a this course my whole life.",
    "I hate this so much!",
]
# 將原始文字轉換成標記
inputs = tokenizer(
  raw_inputs,
  padding=True,         # Activates and controls padding
  truncation=True,      # Activates and controls truncation
  return_tensors="pt",  # Return PyTorch tensor objects
)

pprint(inputs)


{'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0]]),
 'input_ids': tensor([[ 101, 1045, 1005, 2310, 2042, 3403, 2005, 1037, 2023, 2607, 2026, 2878,
         2166, 1012,  102],
        [ 101, 1045, 5223, 2023, 2061, 2172,  999,  102,    0,    0,    0,    0,
            0,    0,    0]]),
 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])}


1. 第二句子長度不夠，所以會被補上 [PAD] token
2. Attention mask 會標記出哪些是 padding 的 token

我們也可以將標記轉換回原始文本，這樣我們就可以看到模型的輸入是什麼樣子的。

In [38]:
tokenizer.decode(inputs["input_ids"][0])

"[CLS] i ' ve been waiting for a this course my whole life. [SEP]"

In [39]:
tokenizer.decode(inputs["input_ids"][1])

'[CLS] i hate this so much! [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

### 多語分詞器 (Multilingual Tokenizer)

如果將此模型應用在中文文本上，會發生什麼狀況呢？

In [40]:
raw_inputs = [
    "我門正在學習目前正夯的變形金剛模型！",
    "你們喜歡這個課程嗎？"
]
# 將原始文字轉換成標記
inputs = tokenizer(
  raw_inputs,
  padding=True,         # Activates and controls padding
  truncation=True,      # Activates and controls truncation
  return_tensors="pt",  # Return PyTorch tensor objects
  )
# 將標記轉換回文字
tokenizer.decode(inputs["input_ids"][1])

'[CLS] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] ？ [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

由於 Tokenizer 是基於英文訓練的，所以它無法正確處理中文文本。這是因為中文文本的分詞方式與英文不同。因此，我們需要使用中文分詞器來處理中文文本。

In [41]:
# 載入 Multilingual BERT tokenizer
model_name = "google-bert/bert-base-multilingual-uncased"
multilingual_tokenizer = AutoTokenizer.from_pretrained(model_name)

In [42]:
# vocab_szie: 30522 > 105879
# model_max_length: 512
multilingual_tokenizer

BertTokenizerFast(name_or_path='google-bert/bert-base-multilingual-uncased', vocab_size=105879, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [43]:
# 將原始文字轉換成標記
multilingual_inputs = tokenizer(
  raw_inputs,
  padding=True,         # Activates and controls padding
  truncation=True,      # Activates and controls truncation
  return_tensors="pt",  # Return PyTorch tensor objects
)
# 將標記轉換回文字
multilingual_tokenizer.decode(multilingual_inputs["input_ids"][1])

'[CLS] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] 傕 [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

## Input Embedding

Transformer 架構首先是 Input Embedding 層，它將輸入序列轉換為維度 $\ d_{model} = 512 $ 的向量。

![Input Embedding](https://www.alexisalulema.com/wp-content/uploads/2022/08/input.embedding.png)

> $\ d_{model} = 512 $ 的值是由架構設計者設定的，這個維度的值可以根據目標進行修改。

### 分詞器 (Tokenizer)

Embedding 從分詞後的文本中獲取數據。對於每個詞，生成一個大小為 $\ d_{model} = 512 $ 的向量。對於意思接近的詞，Embedding 向量應該是相似的。

透過一個小實驗來驗證這個假設：

我們給定一句範例文本，包含多語義的文字 `bank`，這個詞有兩種不同的意思：

```
After stealing money from the bank vault, the bank robber was seen
fishing on the Mississippi river bank.
```

In [44]:
# 給定一句範例文本，包含多語義的文字 'bank'
text = '''
After stealing money from the bank vault, the bank robber was seen
fishing on the Mississippi river bank.
'''

# 將原始文字轉換成標記
tokenized_inputs = tokenizer(
    text,
    padding=True,
    truncation=True,
    return_tensors="pt"
)

將標記轉換回原始文本，我們可以看到 `bank` 這個詞在兩個不同的上下文中卻有相同的標記。請注意，這裡的 token id 是對應到 vocab 的 index，此時 `bank` 這個字具有相同的 token id。

In [45]:
# Display the words with their indeces.
for id in tokenized_inputs["input_ids"][0]:
    pprint('{:<12} {:>6,}'.format(tokenizer.decode(id), id))

'[CLS]           101'
'after         2,044'
'stealing     11,065'
'money         2,769'
'from          2,013'
'the           1,996'
'bank          2,924'
'vault        11,632'
',             1,010'
'the           1,996'
'bank          2,924'
'robber       27,307'
'was           2,001'
'seen          2,464'
'fishing       5,645'
'on            2,006'
'the           1,996'
'mississippi   5,900'
'river         2,314'
'bank          2,924'
'.             1,012'
'[SEP]           102'


此刻，我們已經將原始文本轉換為標記，總計有 22 個標記。這些標記將被送入 Embedding 層。

In [46]:
len(tokenized_inputs["input_ids"][0])

22

### 隱藏狀態（Hidden States）

為了方便我們深入觀察模型，我們透過 `output_hidden_states=True` 參數來獲取模型的隱藏狀態（hidden states），隱藏狀態是模型在每一層計算出的中間表示。

In [47]:
from transformers import AutoModel

# 載入 BERT model
model = AutoModel.from_pretrained(
  model_name,
  output_hidden_states=True,  # Whether the model returns all hidden-states.
)

# 這個 model 有 12 層的 transformer blocks, 及第一層的 embedding 層, 共 13 層
model

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(105879, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=Fals

以下程式碼使用 BERT 模型處理文本，並收集所有層生成的隱藏狀態。這樣可以幫助我們了解模型的結構和輸出數據的維度。

使用 `torch.no_grad()` 禁用梯度計算，這樣可以節省內存並加快計算速度，因為我們只需要前向傳播的結果。

In [48]:
import torch

# Run the text through BERT, and collect all of the hidden states produced from all 12 layers.
with torch.no_grad():
    outputs = model(**tokenized_inputs)
    hidden_states = outputs.hidden_states

In [49]:
n_hidden_states = len(hidden_states)
pprint(f"Number of layers: {n_hidden_states} (initial embeddings + 12 BERT layers)")

'Number of layers: 13 (initial embeddings + 12 BERT layers)'


根據[文件](https://huggingface.co/docs/transformers/model_doc/bert#transformers.BertModel)，每一層的隱藏狀態的維度是 `(batch_size, sequence_length, hidden_size)`，其中 `hidden_size` 是模型的維度，這裡是 768。

In [50]:
# 每一層的張量的尺寸（shape）
hidden_states[0].size()

torch.Size([1, 22, 768])

這個模型的完整隱藏狀態集，存儲在 `hidden_states` 物件中，這個物件有四個維度，順序如下：

1. 層數（13 層）
2. 批次數（1 句話）
3. 詞/標記 (Token) 數（我們句子中的 22 個 Token）
4. 隱藏單元/特徵數（768 個特徵）

In [51]:
n_batches = hidden_states[0].size(0)
n_tokens = hidden_states[0].size(1)
n_hidden_units = hidden_states[0].size(2)

pprint(f"Number of values: {n_hidden_states * n_batches * n_tokens * n_hidden_units}")

'Number of values: 219648'


僅僅為了表示我們的一句話，就需要 219,648 個數值！

### 詞嵌入 (Word Embedding)

由於回傳的是 13 層的張量 `(batch_size, sequence_length, hidden_size)`，我們可以將這些層的張量透過 `torch.stack()` 連接起來，並創建一個新的維度來表示這些層。這樣我們就可以看到每個詞的所有層的隱藏狀態。

In [52]:
# 透過 torch.stack 將 13 層張量串接在一起，並在第 0 維度（dim=0）上
token_embeddings = torch.stack(hidden_states, dim=0)

# 新的張量的尺寸（shape）
token_embeddings.size()

torch.Size([13, 1, 22, 768])

移除 `token_embeddings` 張量中指定維度 `dim=1` 的單維度。相當於移除第一個維度的 `batch_size`。

In [53]:
# Let’s get rid of the “batches” dimension since we don’t need it.
# Remove dimension 1, the "batches".
token_embeddings = torch.squeeze(token_embeddings, dim=1)

# 新的張量的尺寸（shape）
token_embeddings.size()

torch.Size([13, 22, 768])

我們想要計算出每個詞的所有層的隱藏狀態，所以我們需要將 `token_embeddings` 張量的維度重新排列，使得每個詞的所有層的隱藏狀態都在同一個維度中。

In [54]:
# Current dimensions: [# layers, # tokens, # features]
# Desired dimensions: [# tokens, # layers, # features]

# Swap dimensions 0 and 1.
token_embeddings = token_embeddings.permute(1,0,2)

token_embeddings.size()

torch.Size([22, 13, 768])

現在，我們該如何處理這些隱藏狀態呢？我們希望為每個標記 (Token) 獲取單獨的向量，但對於我們的每個輸入標記 (Token)，我們有 13 個長度為 768 的單獨向量。

為了獲取單獨的向量，我們需要結合一些層向量，但是哪一層或哪幾層的組合能提供最佳表示呢？

不幸的是，沒有一個簡單的答案，不過讓我們嘗試通過將最後四層相加來創建詞向量，為每個標記 (Token) 生成一個詞向量。

每個向量的長度將是維持長度 768。

In [55]:
# Stores the token vectors, with shape [22 x 768]
token_vecs_sum = []

# 'token_embeddings' is a [22 x 13 x 768] tensor.

# For each token in the sentence...
for token in token_embeddings:

    # 'token' is a [13 x 768] tensor

    # Sum the vectors from the last four layers.
    sum_vec = torch.sum(token[-4:], dim=0)

    # Use 'sum_vec' to represent 'token'.
    token_vecs_sum.append(sum_vec)

pprint(f'Shape is: {len(token_vecs_sum)} x {len(token_vecs_sum[0])}')

'Shape is: 22 x 768'


為了確認這些向量的值確實是依賴於上下文的，讓我們看看在我們示例句子中「bank」這個詞的不同實例：

「After stealing money from the *bank vault*, the *bank robber* was seen fishing on the Mississippi *river bank*.」

讓我們找出這個示例句子中三個「bank」實例的索引。

In [56]:
for i, id in enumerate(tokenized_inputs["input_ids"][0]):
    pprint(f'{i} {tokenizer.decode(id)}')

'0 [CLS]'
'1 after'
'2 stealing'
'3 money'
'4 from'
'5 the'
'6 bank'
'7 vault'
'8 ,'
'9 the'
'10 bank'
'11 robber'
'12 was'
'13 seen'
'14 fishing'
'15 on'
'16 the'
'17 mississippi'
'18 river'
'19 bank'
'20 .'
'21 [SEP]'


分別坐落在第 6, 10, 19 個標記位置。

In [57]:

# We can try printing out their vectors to compare them.
pprint('First 6 vector values for each instance of "bank".')
pprint('')
pprint(f"bank vault  {token_vecs_sum[6][:6]}")
pprint(f"bank robber {token_vecs_sum[10][:6]}")
pprint(f"river bank  {token_vecs_sum[19][:6]}")


'First 6 vector values for each instance of "bank".'
''
'bank vault  tensor([ 1.9346,  1.7786,  2.5095,  0.0338, -1.0880, -0.3089])'
'bank robber tensor([-0.1687,  1.3765,  2.5958,  0.5758, -1.9220,  0.6940])'
'river bank  tensor([ 0.6635,  1.5705,  0.2651,  0.2089, -0.9681,  1.2516])'


我們可以看到這些值有所不同，但讓我們計算向量之間的餘弦相似度來進行更精確的比較。

餘弦相似度測量高維空間中兩個向量之間的角度的餘弦值。它的取值範圍在 -1 和 1 之間，1 表示兩個向量是相同的，0 表示它們是正交的（即，它們之間沒有相關性）。

In [58]:
from scipy.spatial.distance import cosine

# 比較 'bank robber' 與 'bank vault' 的相似度
same_bank = 1 - cosine(token_vecs_sum[10], token_vecs_sum[6])
pprint('Vector similarity for  *similar*  meanings:  %.2f' % same_bank)

'Vector similarity for  *similar*  meanings:  0.91'


In [59]:
# 比較 'bank robber' 與 'river bank' 的相似度
diff_bank = 1 - cosine(token_vecs_sum[10], token_vecs_sum[19])
pprint('Vector similarity for *different* meanings:  %.2f' % diff_bank)

'Vector similarity for *different* meanings:  0.84'


透過這個小實驗我們觀察到

1. 分詞器 (Tokenizer) 會將相同的詞轉換為相同的標記 (Token)，這是因為字典是固定的。
2. Embedding 層會為每個標記生成一個向量；對於意思相近的詞，Embedding 向量應該是相似的。

## Positional Encoding (PE)

![](https://www.alexisalulema.com/wp-content/uploads/2022/08/positional.encoding.png)

Embedding 向量為 Transformer 提供了大量有關序列中詞語之間關係的信息。然而，仍然需要信息來指示詞語在序列中的位置，這就是位置編碼 (Positional Encoding) 過程的用途。

初始序列中的每個詞都必須具有位置編碼（PE）信息，但由於 Transformer 的主要重點是注意力機制，因此這個向量的生成必須簡單。

這項任務的挑戰在於為位置編碼函數的每個輸出向量生成一個維度為 $\ d_{model} = 512 $ 的向量。架構的作者找到了一種巧妙的方法，使用正弦和餘弦值表示位置編碼。

$\ PE_{\text{sin}}(\text{pos}, 2i) = \sin\left(\frac{\text{pos}}{10000^{\frac{2i}{d_{\text{model}}}}}\right) $

$\ PE_{\text{cos}}(\text{pos}, 2i+1) = \cos\left(\frac{\text{pos}}{10000^{\frac{2i}{d_{\text{model}}}}}\right) $

In [60]:
import math

d_model = 512


def positional_encoding(position):
    pe = [None] * d_model

    for i in range(0, d_model, 2):
        pe[i] = math.sin(position / (10000 ** ((2 * i) / d_model)))
        pe[i + 1] = math.cos(position / (10000 ** ((2 * i) / d_model)))

    return pe

# 計算第 3 個位置的 positional encoding
pprint(positional_encoding(3), compact=True)

[0.1411200080598672, -0.9899924966004454, 0.34278182124536544, -0.9394150430048,
 0.517305716423722, -0.8558006752482378, 0.6624361227092077,
 -0.7491184040790825, 0.7782725224195125, -0.6279266524418035,
 0.8664767242753264, -0.4992174739420687, 0.9296448405538298,
 -0.3684568773026832, 0.9708383734915317, -0.23973496314955228,
 0.993253167134793, -0.11596614150993839, 0.9999995959150344,
 0.0008989826293240337, 0.9939677561484435, 0.10967269367180021,
 0.9777546365250185, 0.2097519266987291, 0.9536344621455892, 0.3009672949147346,
 0.9235576307533468, 0.38345964934692234, 0.8891674026610501,
 0.45758204733687047, 0.8518265889844492, 0.5238238848116019,
 0.8126488966420368, 0.5827536107022249, 0.7725314052467654, 0.6349765569747106,
 0.732185972853175, 0.6811047651845122, 0.692168299382709, 0.7217361327588141,
 0.652904012444876, 0.7574406580936761, 0.614711558412838, 0.7887519888746146,
 0.5778219439143785, 0.8161628520895868, 0.542395526623231, 0.8401232604202243,
 0.508536134419632

最後，詞編碼向量應該加到位置編碼向量上，然後將結果輸入到編碼器/解碼器區塊中：

$\ Embedding_{output} = Embedding_{initial} + PE_{word-position} $